<h1 style="color:red; text-align:center; text-decoration:underline;">Réseaux bayésiens</h1>

<h2 style="color:green; text-decoration:underline;">Test 1 : Décision météo (prendre un parapluie)</h2>

Ce test simule une prise de décision face à l’incertitude météorologique. Un réseau bayésien modélise la probabilité qu’il pleuve en fonction de certaines observations telles que la météo prévue ou l’état du ciel. L’objectif est de déterminer s’il faut emporter un parapluie ou non, en mettant à jour automatiquement les probabilités avec la règle de Bayes.

In [6]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# 1. Définir la structure du graphe
model = DiscreteBayesianNetwork([
    ('Cloudy', 'Rain'),
    ('Rain', 'Report'),
    ('Rain', 'Umbrella')
])

# 2. Définir les CPDs (probabilités conditionnelles)

# P(Cloudy)
cpd_cloudy = TabularCPD(
    variable='Cloudy', variable_card=2,
    values=[[0.5], [0.5]],
    state_names={'Cloudy': ['Non', 'Oui']}
)

# P(Rain | Cloudy)
cpd_rain = TabularCPD(
    variable='Rain', variable_card=2,
    values=[[0.8, 0.2],  # Rain = Non
            [0.2, 0.8]], # Rain = Oui
    evidence=['Cloudy'],
    evidence_card=[2],
    state_names={'Rain': ['Non', 'Oui'], 'Cloudy': ['Non', 'Oui']}
)

# P(Report | Rain)
cpd_report = TabularCPD(
    variable='Report', variable_card=2,
    values=[[0.9, 0.1],  # Report = Non
            [0.1, 0.9]], # Report = Oui
    evidence=['Rain'],
    evidence_card=[2],
    state_names={'Report': ['Non', 'Oui'], 'Rain': ['Non', 'Oui']}
)

# P(Umbrella | Rain)
cpd_umbrella = TabularCPD(
    variable='Umbrella', variable_card=2,
    values=[[0.05, 0.95],  # Umbrella = Non
            [0.95, 0.05]], # Umbrella = Oui
    evidence=['Rain'],
    evidence_card=[2],
    state_names={'Umbrella': ['Non', 'Oui'], 'Rain': ['Non', 'Oui']}
)

# 3. Ajouter les CPDs au modèle
model.add_cpds(cpd_cloudy, cpd_rain, cpd_report, cpd_umbrella)

# 4. Vérifier la validité du modèle
assert model.check_model()

# 5. Inférence
infer = VariableElimination(model)

# 🔍 Cas 1 : La météo dit qu'il va pleuvoir (Report = Oui)
print("🔎 Cas 1 : Report = Oui (prévision météo)")
result1 = infer.query(variables=['Umbrella'], evidence={'Report': 'Oui'})
print(result1)

# 🔍 Cas 2 : Le ciel est nuageux (Cloudy = Oui)
print("\n🔎 Cas 2 : Cloudy = Oui (le ciel est gris)")
result2 = infer.query(variables=['Umbrella'], evidence={'Cloudy': 'Oui'})
print(result2)

# 🔍 Cas 3 : Report = Oui ET Cloudy = Oui
print("\n🔎 Cas 3 : Report = Oui ET Cloudy = Oui")
result3 = infer.query(variables=['Umbrella'], evidence={'Report': 'Oui', 'Cloudy': 'Oui'})
print(result3)

# 🔍 Cas 4 : Aucun indice (aucune évidence) → proba globale
print("\n🔎 Cas 4 : Aucun indice (probabilité globale d’avoir un parapluie)")
result4 = infer.query(variables=['Umbrella'])
print(result4)


🔎 Cas 1 : Report = Oui (prévision météo)
+---------------+-----------------+
| Umbrella      |   phi(Umbrella) |
+===============+=================+
| Umbrella(Non) |          0.8600 |
+---------------+-----------------+
| Umbrella(Oui) |          0.1400 |
+---------------+-----------------+

🔎 Cas 2 : Cloudy = Oui (le ciel est gris)
+---------------+-----------------+
| Umbrella      |   phi(Umbrella) |
+===============+=================+
| Umbrella(Non) |          0.7700 |
+---------------+-----------------+
| Umbrella(Oui) |          0.2300 |
+---------------+-----------------+

🔎 Cas 3 : Report = Oui ET Cloudy = Oui
+---------------+-----------------+
| Umbrella      |   phi(Umbrella) |
+===============+=================+
| Umbrella(Non) |          0.9257 |
+---------------+-----------------+
| Umbrella(Oui) |          0.0743 |
+---------------+-----------------+

🔎 Cas 4 : Aucun indice (probabilité globale d’avoir un parapluie)
+---------------+-----------------+
| Umbrella      |

<h2 style="color:#0056b3; text-decoration:underline;">Résultats</h2>
Dans ce premier test, un réseau bayésien a été utilisé pour modéliser une situation de décision incertaine : faut-il emporter un parapluie ou non ?  
Le modèle intègre divers facteurs contextuels, tels que la météo et l’état du ciel, et met à jour automatiquement les probabilités à l’aide de la règle de Bayes dès qu’une information devient disponible (par exemple, un ciel nuageux).  
Ce type de raisonnement probabiliste permet d’aboutir à une décision rationnelle même en présence d’informations incomplètes ou incertaines.  
Il s’agit d’une approche particulièrement pertinente dans des domaines comme la médecine, les prévisions météorologiques ou l’intelligence artificielle décisionnelle.

<h2 style="color:green; text-decoration:underline;">Test 2 : Détection d'intrusion réseau</h2>

Ce scénario modélise un système intelligent de cybersécurité. Le réseau bayésien évalue la probabilité d’une attaque en fonction de symptômes observés comme une surcharge CPU, des erreurs de connexion ou un trafic inhabituel. Le système met à jour ses estimations automatiquement selon les observations disponibles, même si certaines données sont incomplètes.

In [7]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# 1. Définition de la structure du réseau bayésien
model = DiscreteBayesianNetwork([
    ('Intrusion', 'FailedLogin'),
    ('Intrusion', 'HighTraffic'),
    ('Intrusion', 'CpuOverload'),
    ('FailedLogin', 'OffHoursActivity')
])

# 2. Définition des tables de probabilités conditionnelles (CPD)

cpd_intrusion = TabularCPD(
    variable='Intrusion', variable_card=2,
    values=[[0.95], [0.05]],
    state_names={'Intrusion': ['Non', 'Oui']}
)

cpd_failedlogin = TabularCPD(
    variable='FailedLogin', variable_card=2,
    values=[[0.8, 0.15], [0.2, 0.85]],
    evidence=['Intrusion'],
    evidence_card=[2],
    state_names={'FailedLogin': ['Non', 'Oui'], 'Intrusion': ['Non', 'Oui']}
)

cpd_hightraffic = TabularCPD(
    variable='HighTraffic', variable_card=2,
    values=[[0.7, 0.1], [0.3, 0.9]],
    evidence=['Intrusion'],
    evidence_card=[2],
    state_names={'HighTraffic': ['Non', 'Oui'], 'Intrusion': ['Non', 'Oui']}
)

cpd_cpuoverload = TabularCPD(
    variable='CpuOverload', variable_card=2,
    values=[[0.85, 0.25], [0.15, 0.75]],
    evidence=['Intrusion'],
    evidence_card=[2],
    state_names={'CpuOverload': ['Non', 'Oui'], 'Intrusion': ['Non', 'Oui']}
)

cpd_offhours = TabularCPD(
    variable='OffHoursActivity', variable_card=2,
    values=[[0.6, 0.35], [0.4, 0.65]],
    evidence=['FailedLogin'],
    evidence_card=[2],
    state_names={'OffHoursActivity': ['Non', 'Oui'], 'FailedLogin': ['Non', 'Oui']}
)

# 3. Ajout des CPDs au modèle
model.add_cpds(cpd_intrusion, cpd_failedlogin, cpd_hightraffic, cpd_cpuoverload, cpd_offhours)

# 4. Vérification de la validité du modèle
assert model.check_model()

# 5. Inférence
infer = VariableElimination(model)

# 6. Cas d’analyse avec commentaires explicatifs

print("\n🔍 Cas 1 : CpuOverload = Oui (seulement surcharge du CPU)")
print(infer.query(variables=['Intrusion'], evidence={'CpuOverload': 'Oui'}))

print("\n🔍 Cas 2 : FailedLogin = Oui et OffHoursActivity = Oui")
print(infer.query(variables=['Intrusion'], evidence={'FailedLogin': 'Oui', 'OffHoursActivity': 'Oui'}))

print("\n🔍 Cas 3 : HighTraffic = Oui et CpuOverload = Oui")
print(infer.query(variables=['Intrusion'], evidence={'HighTraffic': 'Oui', 'CpuOverload': 'Oui'}))

print("\n🔍 Cas 4 : Tous les symptômes activés")
print(infer.query(variables=['Intrusion'], evidence={
    'FailedLogin': 'Oui',
    'HighTraffic': 'Oui',
    'CpuOverload': 'Oui',
    'OffHoursActivity': 'Oui'
}))

print("\n🔍 Cas 5 : Aucun symptôme observé (probabilité a priori)")
print(infer.query(variables=['Intrusion']))



🔍 Cas 1 : CpuOverload = Oui (seulement surcharge du CPU)
+----------------+------------------+
| Intrusion      |   phi(Intrusion) |
+================+==================+
| Intrusion(Non) |           0.7917 |
+----------------+------------------+
| Intrusion(Oui) |           0.2083 |
+----------------+------------------+

🔍 Cas 2 : FailedLogin = Oui et OffHoursActivity = Oui
+----------------+------------------+
| Intrusion      |   phi(Intrusion) |
+================+==================+
| Intrusion(Non) |           0.8172 |
+----------------+------------------+
| Intrusion(Oui) |           0.1828 |
+----------------+------------------+

🔍 Cas 3 : HighTraffic = Oui et CpuOverload = Oui
+----------------+------------------+
| Intrusion      |   phi(Intrusion) |
+================+==================+
| Intrusion(Non) |           0.5588 |
+----------------+------------------+
| Intrusion(Oui) |           0.4412 |
+----------------+------------------+

🔍 Cas 4 : Tous les symptômes activés
+

<h2 style="color:#0056b3; text-decoration:underline;">Résultats</h2>
Le second test repose sur la modélisation d’un système de détection d’intrusion à l’aide d’un réseau bayésien.  
À partir d’observations concrètes (surcharge CPU, échecs de connexion, trafic anormal), le modèle calcule la probabilité d’un comportement malveillant en cours.  
Les variables sont reliées par des liens causaux, permettant au modèle d’ajuster dynamiquement ses estimations en fonction des données observées.  
Ce type de modélisation probabiliste est particulièrement adapté aux systèmes de sécurité intelligents, car il conserve son efficacité même en présence de données incomplètes ou bruitées.